In [129]:
import numpy as np
import matplotlib.pyplot as plt
import math
import sympy as sp
from sympy.solvers import solve
import pandas as pd

## Модельная задача

### Аналитическое решение

In [130]:
k = 1. + np.cos(0.5) * np.cos(0.5)
q = 1.
f = np.sin(0.5) * np.sin(0.5)
beta1 = 1.
eps1 = 0
beta2 = 1.
eps2 = -1.

In [131]:
def analytic_solution(k,q,f,x,beta1,eps1,beta2,eps2):
    lmb = lambda q,k: np.sqrt(q/k)
    u = lambda c1,c2,lmb,x,f,q: c1 * np.exp(lmb*x) + c2 * np.exp(-lmb*x) + f/q

    lmb1 = lmb(q,k)
    lmb2 = - lmb1
    c1 = ((k*lmb2 + beta2)*(beta1*f - eps1*q) * np.exp(lmb2) + (k*lmb2 - beta1)*(beta2*f - eps2*q)) / (q * ((k*lmb1 - beta1)*(k*lmb2 + beta2)*np.exp(lmb2) - (k*lmb2 - beta1)*(k*lmb1 + beta2)*np.exp(lmb1)))
    c2 = ((k*lmb1 + beta2)*(beta1*f - eps1*q) * np.exp(lmb1) + (k*lmb1 - beta1)*(beta2*f - eps2*q)) / (q * ((k*lmb2 - beta1)*(k*lmb1 + beta2)*np.exp(lmb1) - (k*lmb1 + beta2)*(k*lmb1 - beta1)*np.exp(lmb2)))
    u1 = u(c1,c2,lmb1,x,f,q)
    return u1

In [132]:
def analytic_points(k,q,f,beta1,eps1,beta2,eps2):
        x1 = np.linspace(0,1,11)
        y1 = []
        for x in x1:
                res1 = analytic_solution(k,q,f,x,beta1,eps1,beta2,eps2)
                y1.append(res1)
        return y1

### Метод прогонки

In [133]:
def progonka_1(n,k,q,f,beta1,eps1,beta2,eps2):
    h = 1/n
    x1 = np.linspace(0,1,11)
    x0 = 1
    solutions = np.zeros(11)
    u = np.zeros(n+1)
    alpha = np.zeros(n+1)
    beta = np.zeros(n+1)
    al = k
    bl = -2*k - q*h**2
    cl = k
    dl = -f*h**2
    a0 = k
    b0 = -k-beta1*h
    c0 = 0
    d0 = -eps1*h
    aL = 0
    bL = -k-beta2*h
    cL=k
    dL = -eps2*h
    
    alpha[0] = -a0/b0; beta[0] = d0/b0
    for i in range(1,n+1):
        alpha[i] = -al/(bl+cl*alpha[i-1])
        beta[i] = (dl - cl*beta[i-1])/(bl+cl*alpha[i-1])

    u[-1] = (dL-cL*beta[-2])/(bL+cL*alpha[-2]); 
    solutions[-1] = u[-1]
    count = 9
    t = 0.9
    
    for i in range(n-1,-1,-1):
        x0 -= h
        u[i] = alpha[i]*u[i+1]+beta[i]
        if abs(x0 - t) < 10**(-10):
            solutions[count] = u[i]
            count -= 1
            t -= 0.1
    
    return solutions

In [134]:
x = np.linspace(0,1,11)
u1 = progonka_1(10,k,q,f,beta1,eps1,beta2,eps2)
a1 = analytic_points(k,q,f,beta1,eps1,beta2,eps2)
delta1 = []; 
for i in range(len(u1)):
    delta1.append(abs(u1[i] - a1[i]))
df = pd.DataFrame({"x": x, "u аналит.": a1, "u числ. ": u1 ,"delta u": delta1})
display(df)
print("delta u max =", max(delta1))

,x,u аналит.,u числ.,delta u
0,0.0,-1.96386223050994,-0.173943,1.78991904696278
1,0.1,-2.08110922395959,-0.183770,1.89733958105845
2,0.2,-2.21141750994839,-0.195933,2.01548477960768
3,0.3,-2.35552357723608,-0.210501,2.14502241922991
4,0.4,-2.51424189819457,-0.227557,2.28668467182434
5,0.5,-2.68846953212034,-0.247197,2.44127224294761
6,0.6,-2.87919119531943,-0.269532,2.60965889757390
7,0.7,-3.08748482662067,-0.294688,2.79279639881799
8,0.8,-3.31452767977289,-0.322808,2.99171988753615
9,0.9,-3.56160297715952,-0.354049,3.20755373321168


delta u max = 3.44151788919579


## Задача с переменными коеффициентами

In [135]:
def progonka_2(n,beta1,eps1,beta2,eps2):
    h = 1/n
    x1 = np.linspace(0,1,11); x0 = 1; x = 0 + h
    solutions = np.zeros(11)
    k = lambda x: 1 + np.cos(x) * np.cos(x)
    q = lambda x: 1
    f = lambda x: np.sin(x) * np.sin(x)
    
    u = np.zeros(n+1)
    alpha = np.zeros(n+1)
    beta = np.zeros(n+1)
    al = lambda x: k(x+h/2)
    bl = lambda x: -(k(x+h/2)+k(x-h/2)+q(x)*h**2)
    cl = lambda x: k(x-h/2)
    dl = lambda x: -f(x)*h**2
    a0 = k(0)
    b0 = -k(0)-beta1*h
    c0 = 0
    d0 = -eps1*h
    aL = 0
    bL = -k(1)-beta2*h
    cL = k(1)
    dL = -eps2*h

    alpha[0] = -a0/b0; beta[0] = d0/b0
    for i in range(1,n+1):
        alpha[i] = -al(x)/(bl(x)+cl(x)*alpha[i-1])
        beta[i] = (dl(x) - cl(x)*beta[i-1])/(bl(x)+cl(x)*alpha[i-1])
        x += h
    u[-1] = (dL-cL*beta[-2])/(bL+cL*alpha[-2]); 
    solutions[-1] = u[-1]
    count = 9
    t = 0.9
    for i in range(n-1,-1,-1):
        x0 -= h
        u[i] = alpha[i]*u[i+1]+beta[i]
        if abs(x0 - t) < 10**(-10):
            solutions[count] = u[i]
            count -= 1
            t -= 0.1
    
    return solutions

In [136]:
a2 = progonka_2(10**6,beta1,eps1,beta2,eps2)

In [137]:
x = np.linspace(0,1,11)
u2 = progonka_2(30,beta1,eps1,beta2,eps2)
delta2 = []
for i in range(len(u2)):
    delta2.append(abs(u2[i] - a2[i]))
df = pd.DataFrame({"x": x, "u аналит.": a2, "u числ. ": u2 ,"delta u ": delta2})
display(df)
print("delta u max =", max(delta2))

,x,u аналит.,u числ.,delta u
0,0.0,-0.152739,-0.159746,0.007006
1,0.1,-0.160783,-0.168022,0.007239
2,0.2,-0.169780,-0.177289,0.007509
3,0.3,-0.180047,-0.187870,0.007823
4,0.4,-0.192053,-0.200239,0.008186
5,0.5,-0.206444,-0.215051,0.008607
6,0.6,-0.224073,-0.233171,0.009098
7,0.7,-0.246042,-0.255712,0.009670
8,0.8,-0.273735,-0.284077,0.010341
9,0.9,-0.308857,-0.319986,0.011129


delta u max = 0.012057087581208326


## Таблица результатов

In [138]:
x = np.linspace(0,1,11)
a1 = analytic_points(k,q,f,beta1,eps1,beta2,eps2)
a2 = progonka_2(10**6,beta1,eps1,beta2,eps2)

In [139]:
delta1 = []
delta2 = []
u1 = progonka_1(20,k,q,f,beta1,eps1,beta2,eps2)
u2 = progonka_2(20,beta1,eps1,beta2,eps2)
for i in range(len(u1)):
    delta1.append(abs(u1[i] - a1[i]))
    delta2.append(abs(u2[i] - a2[i]))
df = pd.DataFrame({"x": x, "u мод. аналит.": a1, "u мод. числ. ": u1 ,"du мод.": delta1, 
                  "u пер. аналит.": a2, "u пер. числ. ": u2 ,"du": delta2})
display(df)
print("du мод. макс =", max(delta1))
print("du перем. макс = ", max(delta2))

,x,u мод. аналит.,u мод. числ.,du мод.,u пер. аналит.,u пер. числ.,du
0,0.0,-1.96386223050994,-0.165724,1.79813810410915,-0.152739,-0.163311,0.010572
1,0.1,-2.08110922395959,-0.175652,1.90545766976458,-0.160783,-0.171700,0.010917
2,0.2,-2.21141750994839,-0.187871,2.02354695204751,-0.169780,-0.181099,0.011319
3,0.3,-2.35552357723608,-0.202450,2.15307338732100,-0.180047,-0.191832,0.011784
4,0.4,-2.51424189819457,-0.219473,2.29476905510751,-0.192053,-0.204377,0.012324
5,0.5,-2.68846953212034,-0.239035,2.44943481578029,-0.206444,-0.219394,0.012950
6,0.6,-2.87919119531943,-0.261246,2.61794483700624,-0.224073,-0.237751,0.013678
7,0.7,-3.08748482662067,-0.286233,2.80125153452335,-0.246042,-0.260570,0.014529
8,0.8,-3.31452767977289,-0.314137,3.00039095517809,-0.273735,-0.289261,0.015525
9,0.9,-3.56160297715952,-0.345114,3.21648863264739,-0.308857,-0.325553,0.016697


du мод. макс = 3.45076594894183
du перем. макс =  0.0180758685614823
